Previously links were created using some string interpolation with Ruby in Jekyll:

```
<img
src="/assets/posts/{{ page.date | date: "%Y-%m-%d" }}-{{ page.title | slugify }}/notifications.png"
alt="GitHub Notifications icon always-on mode"
class="center-aligned"
/>
```

To simplify it, in Hugo I used a shortcode. Now this needs to be converted to:

```
{{< showimage
      image=notifications.png
      alt="GitHub Notifications icon always-on mode"
      caption="GitHub Notifications icon always-on mode"
>}}
```

We don't need the `class` attribute. But other attributes must stop the execution of the conversion, to be considered whether they must be added to the shortcode template or not.

In [1]:
from pathlib import Path
import re
import fileinput
import sys
from bs4 import BeautifulSoup
from string import Template

In [2]:
p = Path('../../content/posts')
# there's always a slugify call, filter by files with that...
subfiles = [x for x in p.iterdir() if x.is_file() and 'src="/assets/posts/{{' in x.read_text()]

In [3]:
print(f'Processing {len(subfiles)} files')

Processing 98 files


In [4]:
IMG_PATTERN = re.compile(r".*(<img.*src=\"\/assets\/posts\/{{.*slugify[^>]*>).*", re.DOTALL|re.MULTILINE)
ALLOWED_ATTRS = ['src', 'alt', 'title', 'class', 'style', 'width', 'height']

for test_file in subfiles:
    print(f'Processing {test_file}...')
    file_name = test_file.resolve()

    template = Template('''{{< showimage
  image="$src"
  alt="$alt"
  caption="$alt"
  style="$style"
>}}



''')


    with fileinput.FileInput(file_name, inplace=True) as f:
        for line in f:
            new_line = line
            matches = IMG_PATTERN.findall(line)
            if matches:
                for match in matches:
                    match = re.sub('{{[^{]+}}', '', match)
                    parsed_html = BeautifulSoup(match)
                    img_tag = parsed_html.find('img')

                    # check attrs
                    attrs = [k for k in img_tag.attrs.keys()]
                    diff = list(set(attrs) - set(ALLOWED_ATTRS))
                    if len(diff) > 0:
                        raise ValueError(f'Found extra unallowed attributes: [{diff}]: {img_tag}')


                    src = img_tag['src']
                    src = src[src.rindex('/') + 1:]
                    if not src:
                        raise ValueError('Missing src!')

                    alt = img_tag.get('alt', '')
                    style = img_tag.get('style', '')
                    width = img_tag.get('width', '')
                    height = img_tag.get('height', '')
                    new_line = template.substitute(
                        src=src,
                        alt=alt,
                        style=style,
                        width=width,
                        height=height
                    )

            sys.stdout.write(new_line)

            
print("OK!")

Processing ../../content/posts/2020-03-26-jenkins-active-choices-with-jupyter-notebooks.md...
Processing ../../content/posts/2018-07-10-import-error-when-debugging-cylc-in-eclipse.md...
Processing ../../content/posts/2021-04-06-random-traffic-light.md...
Processing ../../content/posts/2016-07-12-processing-vaisala-radiosonde-data-with-python-and-creating-gruan-like-netcdf-files.md...
Processing ../../content/posts/2018-05-27-what-happens-when-you-upload-a-turtle-file-in-apache-jena-fuseki.md...
Processing ../../content/posts/2010-12-10-article-about-testlink-and-hudson-integration-published.md...
Processing ../../content/posts/2017-01-13-drawing-cave.md...
Processing ../../content/posts/2016-02-27-learning-afl-and-testing-mapserver.md...
Processing ../../content/posts/2016-08-24-revamping-frege-logo-part-2.md...
Processing ../../content/posts/2018-05-13-drawing-santos-dumonts-encantada-in-krita.md...
Processing ../../content/posts/2018-12-22-running-cylc-tasks-on-pbs-torque-with-docker

In [5]:
subfiles

[PosixPath('../../content/posts/2020-03-26-jenkins-active-choices-with-jupyter-notebooks.md'),
 PosixPath('../../content/posts/2018-07-10-import-error-when-debugging-cylc-in-eclipse.md'),
 PosixPath('../../content/posts/2021-04-06-random-traffic-light.md'),
 PosixPath('../../content/posts/2016-07-12-processing-vaisala-radiosonde-data-with-python-and-creating-gruan-like-netcdf-files.md'),
 PosixPath('../../content/posts/2018-05-27-what-happens-when-you-upload-a-turtle-file-in-apache-jena-fuseki.md'),
 PosixPath('../../content/posts/2010-12-10-article-about-testlink-and-hudson-integration-published.md'),
 PosixPath('../../content/posts/2017-01-13-drawing-cave.md'),
 PosixPath('../../content/posts/2016-02-27-learning-afl-and-testing-mapserver.md'),
 PosixPath('../../content/posts/2016-08-24-revamping-frege-logo-part-2.md'),
 PosixPath('../../content/posts/2018-05-13-drawing-santos-dumonts-encantada-in-krita.md'),
 PosixPath('../../content/posts/2018-12-22-running-cylc-tasks-on-pbs-torque-